# Symbolic Linearisation of Quadcopter Model

In [1]:
import sympy as sp
from sympy import sin, cos, tan

In [2]:
x, y, z, u, v, w = sp.symbols('x y z u v w')
phi, theta, psi = sp.symbols('phi theta psi')
p, q, r = sp.symbols('p q r')
u1, u2, u3, u4 = sp.symbols('u1 u2 u3 u4')

In [3]:
m, g = sp.symbols('m g')
Ix, Iy, Iz = sp.symbols('I_x I_y I_z')

In [4]:
state = sp.Matrix([x,y,z,u,v,w,phi,theta,psi,p,q,r])
inputs = sp.Matrix([u1,u2,u3,u4])

In [5]:
sp.print_latex(inputs.transpose())

\left[\begin{matrix}u_{1} & u_{2} & u_{3} & u_{4}\end{matrix}\right]


In [6]:
# Translational Dynamics
dx = u*(cos(theta)*cos(psi)) + v*(sin(phi)*sin(theta)*cos(psi) - cos(phi)*sin(psi)) + w*(cos(phi)*sin(theta)*cos(psi) + sin(phi)*sin(psi))
dy = u*(cos(theta)*sin(psi)) + v*(sin(phi)*sin(theta)*sin(psi) + cos(phi)*cos(psi)) + w*(cos(phi)*sin(theta)*sin(psi) - sin(phi)*cos(psi))
dz = u*(sin(theta)) - v*(sin(phi)*cos(theta)) - w*(cos(phi)*cos(theta))
du = r*v - q*w - g*sin(theta)
dv = p*w - r*u + g*cos(theta)*sin(phi)
dw = q*u - p*v + g*cos(theta)*cos(phi) - u1/m

In [7]:
# Rotational Dynamics
dphi = p + q*(sin(phi)*tan(theta)) + r*(cos(phi)*tan(theta))
dtheta = q*cos(phi) - r*sin(phi)
dpsi = (q*sin(phi) + r*cos(phi))/cos(theta)
dp = (Iy-Iz)/Ix *q*r + u2/Ix
dq = (Iz-Ix)/Iy *p*r + u3/Iy
dr = (Ix-Iy)/Iz *p*q + u4/Iz

In [8]:
f = sp.Matrix([dx,dy,dz,du,dv,dw,dphi,dtheta,dpsi,dp,dq,dr])

Asym = f.jacobian(state)
Bsym = f.jacobian(inputs)

In [9]:
sp.print_latex(f)

\left[\begin{matrix}u \cos{\left(\psi \right)} \cos{\left(\theta \right)} + v \left(\sin{\left(\phi \right)} \sin{\left(\theta \right)} \cos{\left(\psi \right)} - \sin{\left(\psi \right)} \cos{\left(\phi \right)}\right) + w \left(\sin{\left(\phi \right)} \sin{\left(\psi \right)} + \sin{\left(\theta \right)} \cos{\left(\phi \right)} \cos{\left(\psi \right)}\right)\\u \sin{\left(\psi \right)} \cos{\left(\theta \right)} + v \left(\sin{\left(\phi \right)} \sin{\left(\psi \right)} \sin{\left(\theta \right)} + \cos{\left(\phi \right)} \cos{\left(\psi \right)}\right) + w \left(- \sin{\left(\phi \right)} \cos{\left(\psi \right)} + \sin{\left(\psi \right)} \sin{\left(\theta \right)} \cos{\left(\phi \right)}\right)\\u \sin{\left(\theta \right)} - v \sin{\left(\phi \right)} \cos{\left(\theta \right)} - w \cos{\left(\phi \right)} \cos{\left(\theta \right)}\\- g \sin{\left(\theta \right)} - q w + r v\\g \sin{\left(\phi \right)} \cos{\left(\theta \right)} + p w - r u\\g \cos{\left(\phi \right)} \cos

In [10]:
Asym
#Asym

Matrix([
[0, 0, 0, cos(psi)*cos(theta), sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi),  sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi),  v*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi)) + w*(-sin(phi)*sin(theta)*cos(psi) + sin(psi)*cos(phi)), -u*sin(theta)*cos(psi) + v*sin(phi)*cos(psi)*cos(theta) + w*cos(phi)*cos(psi)*cos(theta), -u*sin(psi)*cos(theta) + v*(-sin(phi)*sin(psi)*sin(theta) - cos(phi)*cos(psi)) + w*(sin(phi)*cos(psi) - sin(psi)*sin(theta)*cos(phi)),                  0,                   0,                   0],
[0, 0, 0, sin(psi)*cos(theta), sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi), -sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi), v*(-sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi)) + w*(-sin(phi)*sin(psi)*sin(theta) - cos(phi)*cos(psi)), -u*sin(psi)*sin(theta) + v*sin(phi)*sin(psi)*cos(theta) + w*sin(psi)*cos(phi)*cos(theta),   u*cos(psi)*cos(theta) + v*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + w*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos

In [11]:
Bsym

Matrix([
[   0,     0,     0,     0],
[   0,     0,     0,     0],
[   0,     0,     0,     0],
[   0,     0,     0,     0],
[   0,     0,     0,     0],
[-1/m,     0,     0,     0],
[   0,     0,     0,     0],
[   0,     0,     0,     0],
[   0,     0,     0,     0],
[   0, 1/I_x,     0,     0],
[   0,     0, 1/I_y,     0],
[   0,     0,     0, 1/I_z]])

In [12]:
# Hover equilibrium point

substitutions = {
    u: 0, v: 0, w: 0,
    phi: 0, theta: 0, psi: 0,
    p: 0, q: 0, r: 0,
    u1: -m*g, u2: 0, u3: 0, u4: 0,
}
A = Asym.subs(substitutions)
B = Bsym.subs(substitutions)

In [13]:
A = sp.simplify(A)
B = sp.simplify(B)

In [14]:
A

Matrix([
[0, 0, 0, 1, 0,  0, 0,  0, 0, 0, 0, 0],
[0, 0, 0, 0, 1,  0, 0,  0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, -1, 0,  0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, 0, -g, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, g,  0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, 0,  0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, 0,  0, 0, 1, 0, 0],
[0, 0, 0, 0, 0,  0, 0,  0, 0, 0, 1, 0],
[0, 0, 0, 0, 0,  0, 0,  0, 0, 0, 0, 1],
[0, 0, 0, 0, 0,  0, 0,  0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, 0,  0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, 0,  0, 0, 0, 0, 0]])

In [15]:
sp.print_latex(B)

\left[\begin{matrix}0 & 0 & 0 & 0\\0 & 0 & 0 & 0\\0 & 0 & 0 & 0\\0 & 0 & 0 & 0\\0 & 0 & 0 & 0\\- \frac{1}{m} & 0 & 0 & 0\\0 & 0 & 0 & 0\\0 & 0 & 0 & 0\\0 & 0 & 0 & 0\\0 & \frac{1}{I_{x}} & 0 & 0\\0 & 0 & \frac{1}{I_{y}} & 0\\0 & 0 & 0 & \frac{1}{I_{z}}\end{matrix}\right]


In [16]:
sp.print_latex(A @ state + B @ inputs)

\left[\begin{matrix}u\\v\\- w\\- g \theta\\g \phi\\- \frac{u_{1}}{m}\\p\\q\\r\\\frac{u_{2}}{I_{x}}\\\frac{u_{3}}{I_{y}}\\\frac{u_{4}}{I_{z}}\end{matrix}\right]


In [17]:
Asym

Matrix([
[0, 0, 0, cos(psi)*cos(theta), sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi),  sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi),  v*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi)) + w*(-sin(phi)*sin(theta)*cos(psi) + sin(psi)*cos(phi)), -u*sin(theta)*cos(psi) + v*sin(phi)*cos(psi)*cos(theta) + w*cos(phi)*cos(psi)*cos(theta), -u*sin(psi)*cos(theta) + v*(-sin(phi)*sin(psi)*sin(theta) - cos(phi)*cos(psi)) + w*(sin(phi)*cos(psi) - sin(psi)*sin(theta)*cos(phi)),                  0,                   0,                   0],
[0, 0, 0, sin(psi)*cos(theta), sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi), -sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi), v*(-sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi)) + w*(-sin(phi)*sin(psi)*sin(theta) - cos(phi)*cos(psi)), -u*sin(psi)*sin(theta) + v*sin(phi)*sin(psi)*cos(theta) + w*sin(psi)*cos(phi)*cos(theta),   u*cos(psi)*cos(theta) + v*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + w*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos